# DRIVING STRAIGHT
María Rodríguez Palomo

#### LIBRARIES

In [1]:
import glob
import os
import sys
import sys
import carla
import numpy as np
import random
import time
import cv2
import math

#### VEHICLE CREATION

In [4]:
client = carla.Client('localhost', 2000)
client.set_timeout(2000)
# We will use the world that is created by default
world = client.get_world()
blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('*mini*'))

if bp.has_attribute('color'):
    color = bp.get_attribute('color').recommended_values[2]
    bp.set_attribute('color', '255,0,255')
        
start_point = world.get_map().get_spawn_points()[0]
vehicle = world.spawn_actor(bp, start_point)

#### SENSOR CAMERA CREATION

In [5]:
CAMERA_POS_Z = 3
CAMERA_POS_X = -5
camera_bp=world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x','640')
camera_bp.set_attribute('image_size_y','360')
camera_init_trans= carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
camera=world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data):
    data['image']=np.reshape(np.copy(image.raw_data),(image.height,image.width,4))
    
image_w= camera_bp.get_attribute('image_size_x').as_int()
image_h= camera_bp.get_attribute('image_size_y').as_int()
camera_data = {'image':np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

#### MAINTAINING A CONSTANT SPEED

In [6]:
def maintain_speed(s,preferred_speed,speed_threshold):
    #Return is the acceleration
    if s>=preferred_speed:
        return 0
    elif s < preferred_speed - speed_threshold:
        return 0.8
    else:
        return 0.4

#### DEVELOPER VIEW TAB

In [ ]:
preferred_speed = 20
speed_threshold = 2
steering_angle = 0

#Camera
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
quit=False

while True:
    #Camera
    world.tick()
    if cv2.waitKey(1)==ord('q'): #If we press the q key, the tab disappears
        quit=True
        break
    image = camera_data['image']
    
    velocity = vehicle.get_velocity()
    speed = round(3.6 *math.sqrt(velocity.x**2+ velocity.y**2+velocity.z**2),0)
    image=cv2.putText(image,'Speed: '+str(int(speed))+' km/h', (30,50),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1,cv2.LINE_AA)
    estimated_throttle = maintain_speed(speed,preferred_speed,speed_threshold)
    vehicle.apply_control(carla.VehicleControl(throttle= estimated_throttle, steer=steering_angle))
    cv2.imshow('RGB Camera', image)
cv2.destroyAllWindows()
camera.stop() 

#### DESTROY

In [3]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()